In [1]:
import requests
import json
import mrigutilities as mu
import datetime
import pandas as pd
import calendar
import yfinance as yf

C:\anaconda\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [36]:
def get_expiries(symbol,month,year):
    sql = '''
    select distinct expiry from futures_options_history where symbol='%s' and expiry >= '%s' and expiry <= '%s'
    order by expiry asc
    '''
    engine = mu.sql_engine()
    dates = pd.read_sql(sql%(symbol,datetime.date(year,month,1).strftime('%Y%m%d'),
                                (datetime.date(year,month,calendar.monthrange(year,month)[1])).strftime('%Y%m%d')),engine)
    
    
    dates = sorted(list(dates['expiry']))
    weeks = {}
    for date in dates:
        week = date.isocalendar().week
        if week in weeks.keys():
            weeks[week].append(date)
        else:
            weeks[week] = [date]
    dates = []
    for key in weeks.keys():
        if len(weeks[key]) > 1:
            for date in weeks[key]:
                if date.weekday() == 3:
                    weeks[key] = [date]
            dates.append(weeks[key][-1])
        else:
            dates.append(weeks[key][-1])
    
#     print(dates)
#     print(dates)        
    return dates

                        
                        
def backtest(index,month,year):
    desired_income = 200000
    collateral = 4000000
    margin_per_call_put_pair = 115000
    lot = 50
    print('**************YEAR {}*****MONTH {} *******'.format(str(year),str(month)))
    dates = get_expiries(index,month,year)
    month_expiry = dates[-1]
    nifty_expiry_1,nifty_expiry_2,nifty_expiry_3= dates[0],dates[1],dates[2]
    if len(dates) > 4:
        nifty_expiry_4 = dates[-2]
    else:
        nifty_expiry_4 = None
    
#     month_expiry = mu.last_thursday_of_month(datetime.date(year,month,1))
#     print(month_expiry)
#     nifty_expiry_3 = month_expiry - datetime.timedelta(days=7)
#     nifty_expiry_2 = nifty_expiry_3 - datetime.timedelta(days=7)
#     nifty_expiry_1 = nifty_expiry_2 - datetime.timedelta(days=7)
    print(nifty_expiry_1,nifty_expiry_2,nifty_expiry_3,nifty_expiry_4,month_expiry)
    
#     banknifty_expiry_3 = month_expiry - datetime.timedelta(days=8)
#     banknifty_expiry_2 = banknifty_expiry_3 - datetime.timedelta(days=7)
#     banknifty_expiry_1 = banknifty_expiry_2 - datetime.timedelta(days=7)
#     print(banknifty_expiry_1,banknifty_expiry_2,banknifty_expiry_3,month_expiry)
    
    oc = mu.getOptionData(index,nifty_expiry_1,month_expiry+datetime.timedelta(days=1))
    if not oc.empty:
        oc = oc[(oc['close'] >= 0.001) &
                (oc['open'] >= 0.001) &
                (oc['high'] >= 0.001) &
                (oc['low'] >= 0.001) &
                (oc['oi'] >= 10000) 
               ]
        

        #         oc.index = pd.to_datetime(oc.index).dt.date
        trade_date_position = pd.bdate_range(nifty_expiry_1 - datetime.timedelta(days=6), nifty_expiry_1)[0].date()
        trade_date_position = get_expiries(index,trade_date_position.month,trade_date_position.year)[-1]
        print('TRADE POSITION   {}'.format(trade_date_position))
#         print(oc_trade_date[(oc_trade_date['close'] >= 100) &
#                                     (oc_trade_date['option_type'] == 'CE') &
#                                     (oc_trade_date['expiry'] == month_expiry)])
#         print(oc_trade_date[(oc_trade_date['close'] >= 100) &
#                                     (oc_trade_date['option_type'] == 'PE') &
#                                     (oc_trade_date['expiry'] == month_expiry)])        
#         print(oc_trade_date)
        call_strike,put_strike = -1,-1
        try:
            oc_trade_date = oc.loc[trade_date_position]
            call_strike = sorted(oc_trade_date[(oc_trade_date['close'] >= 100) &
                                        (oc_trade_date['option_type'] == 'CE') &
                                        (oc_trade_date['expiry'] == month_expiry)]['strike'].values)[-1]
            put_strike = sorted(oc_trade_date[(oc_trade_date['close'] >= 100) &
                                        (oc_trade_date['option_type'] == 'PE') &
                                        (oc_trade_date['expiry'] == month_expiry)]['strike'].values)[0]
        except:
            pass
        
        print(call_strike,put_strike)

        if ((call_strike < put_strike) | (call_strike < 0) | (put_strike < 0)) :
            print('Exiting ',month,'   Simulation')
            return {}
        oc = oc[oc['strike'].isin([call_strike,call_strike+200,put_strike,put_strike-200])]
#         print(oc.index)
    
        oc_trade_date = oc.loc[trade_date_position]
        oc_trade_date_final = oc.loc[month_expiry]
#         print(oc_trade_date)
        c1 = {'strike':call_strike,'qty' : -1000,
              'cost': oc_trade_date[(oc_trade_date['strike'] == call_strike) &
                                    (oc_trade_date['option_type'] == 'CE') &
                                    (oc_trade_date['expiry'] == month_expiry)]['close'].values[0],
              'final': oc_trade_date_final[(oc_trade_date_final['strike'] == call_strike) &
                                    (oc_trade_date_final['option_type'] == 'CE') &
                                    (oc_trade_date_final['expiry'] == month_expiry)]['close'].values[0],              
              'exp' : month_expiry}
        p1 = {'strike':put_strike,'qty' : -1000,
              'cost': oc_trade_date[(oc_trade_date['strike'] == put_strike) & 
                                    (oc_trade_date['option_type'] == 'PE')&
                                    (oc_trade_date['expiry'] == month_expiry)]['close'].values[0],
              'final': oc_trade_date_final[(oc_trade_date_final['strike'] == put_strike) & 
                                    (oc_trade_date_final['option_type'] == 'PE')&
                                    (oc_trade_date_final['expiry'] == month_expiry)]['close'].values[0],              
              'exp' : month_expiry}
#         print(c1,p1)
        condor = {'c1':c1,'p1':p1}
        premium = -1*(condor['c1']['qty']*condor['c1']['cost'] + condor['p1']['qty']*condor['p1']['cost'])
        condor_final = -1*((condor['c1']['qty']*condor['c1']['final'] + condor['p1']['qty']*condor['p1']['final']))
        condor_pnl = premium - condor_final 
        
        print('condor        ',condor)
        print('premium       ',premium)
        #         print(premium)

        #************ HEDGE 1 **********
        
        oc_trade_date_final = oc.loc[nifty_expiry_1]
        
        c1 = {'strike':call_strike+200,'qty' : 1000,
              'cost': oc_trade_date[(oc_trade_date['strike'] == call_strike+200) &
                                    (oc_trade_date['option_type'] == 'CE') &
                                    (oc_trade_date['expiry'] == nifty_expiry_1)]['close'].values[0],
              'final': oc_trade_date_final[(oc_trade_date_final['strike'] == call_strike+200) &
                                    (oc_trade_date_final['option_type'] == 'CE') &
                                    (oc_trade_date_final['expiry'] == nifty_expiry_1)]['close'].values[0],
              
              'exp' : nifty_expiry_1}
        p1 = {'strike':put_strike-200,'qty' : 1000,
              'cost': oc_trade_date[(oc_trade_date['strike'] == put_strike-200) & 
                                    (oc_trade_date['option_type'] == 'PE')&
                                    (oc_trade_date['expiry'] == nifty_expiry_1)]['close'].values[0],
              'final': oc_trade_date_final[(oc_trade_date_final['strike'] == put_strike-200) & 
                                    (oc_trade_date_final['option_type'] == 'PE')&
                                    (oc_trade_date_final['expiry'] == nifty_expiry_1)]['close'].values[0],
              
              'exp' : nifty_expiry_1}
        
#         print(c1,p1)
        hedge1 = {'c1':c1,'p1':p1}  
        hedge1_cost = (hedge1['c1']['qty']*hedge1['c1']['cost'] + hedge1['p1']['qty']*hedge1['p1']['cost'])
        premium = premium - hedge1_cost
        hedge1_final = (hedge1['c1']['qty']*hedge1['c1']['final'] + hedge1['p1']['qty']*hedge1['p1']['final'])
        hedge1_pnl = hedge1_final - hedge1_cost
        premium = premium + hedge1_pnl
        
        print('hedge1      ',hedge1)
        print('hedge1_cost       ',hedge1_cost)
        print('hedge1_pnl       ',hedge1_pnl)
        
        print('premium       ',premium)
        
        #************ HEDGE 2 **********
        oc_trade_date = oc.loc[nifty_expiry_1]
        oc_trade_date_final = oc.loc[nifty_expiry_2]
    
        c1 = {'strike':call_strike+200,'qty' : 1000,
              'cost': oc_trade_date[(oc_trade_date['strike'] == call_strike+200) &
                                    (oc_trade_date['option_type'] == 'CE') &
                                    (oc_trade_date['expiry'] == nifty_expiry_1)]['close'].values[0],
              'final': oc_trade_date_final[(oc_trade_date_final['strike'] == call_strike+200) &
                                    (oc_trade_date_final['option_type'] == 'CE') &
                                    (oc_trade_date_final['expiry'] == nifty_expiry_2)]['close'].values[0],              
              'exp' : nifty_expiry_2}
        p1 = {'strike':put_strike-200,'qty' : 1000,
              'cost': oc_trade_date[(oc_trade_date['strike'] == put_strike-200) & 
                                    (oc_trade_date['option_type'] == 'PE')&
                                    (oc_trade_date['expiry'] == nifty_expiry_1)]['close'].values[0],
              'final': oc_trade_date_final[(oc_trade_date_final['strike'] == put_strike-200) & 
                                    (oc_trade_date_final['option_type'] == 'PE')&
                                    (oc_trade_date_final['expiry'] == nifty_expiry_2)]['close'].values[0],              
              'exp' : nifty_expiry_2}
        
#         print(c1,p1)
        hedge2 = {'c1':c1,'p1':p1}   
        hedge2_cost = (hedge2['c1']['qty']*hedge2['c1']['cost'] + hedge2['p1']['qty']*hedge2['p1']['cost'])
        premium = premium - hedge2_cost
        hedge2_final = (hedge2['c1']['qty']*hedge2['c1']['final'] + hedge2['p1']['qty']*hedge2['p1']['final'])
        hedge2_pnl = hedge2_final - hedge2_cost
        premium = premium + hedge2_pnl
        
        print('hedge2      ',hedge2)
        print('hedge2_cost       ',hedge2_cost)
        print('hedge2_pnl       ',hedge2_pnl)
        
        print('premium       ',premium)
        
        #************ HEDGE 3 **********
        
        oc_trade_date = oc.loc[nifty_expiry_2]
        oc_trade_date_final = oc.loc[nifty_expiry_3]
        
    
        c1 = {'strike':call_strike+200,'qty' : 1000,
              'cost': oc_trade_date[(oc_trade_date['strike'] == call_strike+200) &
                                    (oc_trade_date['option_type'] == 'CE') &
                                    (oc_trade_date['expiry'] == nifty_expiry_3)]['close'].values[0],
              'final': oc_trade_date_final[(oc_trade_date_final['strike'] == call_strike+200) &
                                    (oc_trade_date_final['option_type'] == 'CE') &
                                    (oc_trade_date_final['expiry'] == nifty_expiry_3)]['close'].values[0],              
              'exp' : nifty_expiry_3}
        p1 = {'strike':put_strike-200,'qty' : 1000,
              'cost': oc_trade_date[(oc_trade_date['strike'] == put_strike-200) & 
                                    (oc_trade_date['option_type'] == 'PE')&
                                    (oc_trade_date['expiry'] == nifty_expiry_3)]['close'].values[0],
              'final': oc_trade_date_final[(oc_trade_date_final['strike'] == put_strike-200) & 
                                    (oc_trade_date_final['option_type'] == 'PE')&
                                    (oc_trade_date_final['expiry'] == nifty_expiry_3)]['close'].values[0],              
              'exp' : nifty_expiry_3}
        
#         print(c1,p1)
        hedge3 = {'c1':c1,'p1':p1}   
        hedge3_cost = (hedge3['c1']['qty']*hedge3['c1']['cost'] + hedge3['p1']['qty']*hedge3['p1']['cost'])
        premium = premium - hedge3_cost
        hedge3_final = (hedge3['c1']['qty']*hedge3['c1']['final'] + hedge3['p1']['qty']*hedge3['p1']['final'])
        hedge3_pnl = hedge3_final - hedge3_cost
        premium = premium + hedge3_pnl
        
        print('hedge3      ',hedge3)
        print('hedge3_cost       ',hedge3_cost)
        print('hedge3_pnl       ',hedge3_pnl)
        
        print('premium       ',premium)        
        
        
        #************ HEDGE 4 **********
        if nifty_expiry_4 is not None:
            oc_trade_date = oc.loc[nifty_expiry_3]
            oc_trade_date_final = oc.loc[nifty_expiry_4]


            c1 = {'strike':call_strike+200,'qty' : 1000,
                  'cost': oc_trade_date[(oc_trade_date['strike'] == call_strike+200) &
                                        (oc_trade_date['option_type'] == 'CE') &
                                        (oc_trade_date['expiry'] == nifty_expiry_4)]['close'].values[0],
                  'final': oc_trade_date_final[(oc_trade_date_final['strike'] == call_strike+200) &
                                        (oc_trade_date_final['option_type'] == 'CE') &
                                        (oc_trade_date_final['expiry'] == nifty_expiry_4)]['close'].values[0],              
                  'exp' : nifty_expiry_4}
            p1 = {'strike':put_strike-200,'qty' : 1000,
                  'cost': oc_trade_date[(oc_trade_date['strike'] == put_strike-200) & 
                                        (oc_trade_date['option_type'] == 'PE')&
                                        (oc_trade_date['expiry'] == nifty_expiry_4)]['close'].values[0],
                  'final': oc_trade_date_final[(oc_trade_date_final['strike'] == put_strike-200) & 
                                        (oc_trade_date_final['option_type'] == 'PE')&
                                        (oc_trade_date_final['expiry'] == nifty_expiry_4)]['close'].values[0],              
                  'exp' : nifty_expiry_4}

    #         print(c1,p1)
            hedge4 = {'c1':c1,'p1':p1}   
            hedge4_cost = (hedge4['c1']['qty']*hedge4['c1']['cost'] + hedge4['p1']['qty']*hedge4['p1']['cost'])
            premium = premium - hedge4_cost
            hedge4_final = (hedge4['c1']['qty']*hedge4['c1']['final'] + hedge4['p1']['qty']*hedge4['p1']['final'])
            hedge4_pnl = hedge4_final - hedge4_cost
            premium = premium + hedge4_pnl

            print('hedge4      ',hedge4)
            print('hedge4_cost       ',hedge4_cost)
            print('hedge4_pnl       ',hedge4_pnl)

            print('premium       ',premium)            

            #************ HEDGE 4 **********

            oc_trade_date = oc.loc[nifty_expiry_4]
            oc_trade_date_final = oc.loc[month_expiry]


            c1 = {'strike':call_strike+200,'qty' : 1000,
                  'cost': oc_trade_date[(oc_trade_date['strike'] == call_strike+200) &
                                        (oc_trade_date['option_type'] == 'CE') &
                                        (oc_trade_date['expiry'] == month_expiry)]['close'].values[0],
                  'final': oc_trade_date_final[(oc_trade_date_final['strike'] == call_strike+200) &
                                        (oc_trade_date_final['option_type'] == 'CE') &
                                        (oc_trade_date_final['expiry'] == month_expiry)]['close'].values[0],              
                  'exp' : month_expiry}
            p1 = {'strike':put_strike-200,'qty' : 1000,
                  'cost': oc_trade_date[(oc_trade_date['strike'] == put_strike-200) & 
                                        (oc_trade_date['option_type'] == 'PE')&
                                        (oc_trade_date['expiry'] == month_expiry)]['close'].values[0],
                  'final': oc_trade_date_final[(oc_trade_date_final['strike'] == put_strike-200) & 
                                        (oc_trade_date_final['option_type'] == 'PE')&
                                        (oc_trade_date_final['expiry'] == month_expiry)]['close'].values[0],              
                  'exp' : month_expiry}

    #         print(c1,p1)
            hedge5 = {'c1':c1,'p1':p1}   
            hedge5_cost = (hedge5['c1']['qty']*hedge5['c1']['cost'] + hedge5['p1']['qty']*hedge5['p1']['cost'])
            premium = premium - hedge5_cost
            hedge5_final = (hedge5['c1']['qty']*hedge5['c1']['final'] + hedge5['p1']['qty']*hedge5['p1']['final'])
            hedge5_pnl = hedge5_final - hedge5_cost
            premium = premium + hedge5_pnl
            
            print('hedge5      ',hedge5)
            print('hedge5_cost       ',hedge5_cost)
            print('hedge5_pnl       ',hedge5_pnl)            
        else:
            oc_trade_date = oc.loc[nifty_expiry_3]
            oc_trade_date_final = oc.loc[month_expiry]


            c1 = {'strike':call_strike+200,'qty' : 1000,
                  'cost': oc_trade_date[(oc_trade_date['strike'] == call_strike+200) &
                                        (oc_trade_date['option_type'] == 'CE') &
                                        (oc_trade_date['expiry'] == month_expiry)]['close'].values[0],
                  'final': oc_trade_date_final[(oc_trade_date_final['strike'] == call_strike+200) &
                                        (oc_trade_date_final['option_type'] == 'CE') &
                                        (oc_trade_date_final['expiry'] == month_expiry)]['close'].values[0],              
                  'exp' : month_expiry}
            p1 = {'strike':put_strike-200,'qty' : 1000,
                  'cost': oc_trade_date[(oc_trade_date['strike'] == put_strike-200) & 
                                        (oc_trade_date['option_type'] == 'PE')&
                                        (oc_trade_date['expiry'] == month_expiry)]['close'].values[0],
                  'final': oc_trade_date_final[(oc_trade_date_final['strike'] == put_strike-200) & 
                                        (oc_trade_date_final['option_type'] == 'PE')&
                                        (oc_trade_date_final['expiry'] == month_expiry)]['close'].values[0],              
                  'exp' : month_expiry}

    #         print(c1,p1)
            hedge4 = {'c1':c1,'p1':p1}   
            hedge4_cost = (hedge4['c1']['qty']*hedge4['c1']['cost'] + hedge4['p1']['qty']*hedge4['p1']['cost'])
            premium = premium - hedge4_cost
            hedge4_final = (hedge4['c1']['qty']*hedge4['c1']['final'] + hedge4['p1']['qty']*hedge4['p1']['final'])
            hedge4_pnl = hedge4_final - hedge4_cost
            premium = premium + hedge4_pnl
            
            print('hedge4      ',hedge4)
            print('hedge4_cost       ',hedge4_cost)
            print('hedge4_pnl       ',hedge4_pnl)

            print('premium       ',premium)            
            
            
        hedge_pnl = hedge1_pnl+hedge2_pnl+hedge3_pnl+hedge4_pnl+(hedge5_pnl if nifty_expiry_4 is not None else 0 )
        print('premium       ',premium) 
        print('condor_final',condor_final)
        print('condor_pnl      ',condor_pnl)
        print('hedge_pnl', hedge_pnl)
        return {'condor' : condor,
                'condor_pnl' : condor_pnl,
                'hedge_pnl' : hedge_pnl,
                'total_pnl' : condor_pnl+hedge_pnl}

In [38]:
# backtest('NIFTY',12,2023)
start_date = datetime.date(2023,7,1)
end_date = datetime.date(2024,3,1)
date = start_date
result = {}
while date < end_date:
    month = date.month
    year = date.year
#     try:
    res = backtest('NIFTY',month,year)
    result[str(month)+'_'+str(year)] = res
#     except:
#         pass    
    date = date + datetime.timedelta(days = 31)




**************YEAR 2023*****MONTH 7 *******
2023-07-06 2023-07-13 2023-07-20 None 2023-07-27
TRADE POSITION   2023-06-29
-1 -1
Exiting  7    Simulation
**************YEAR 2023*****MONTH 8 *******
2023-08-03 2023-08-10 2023-08-17 2023-08-24 2023-08-31
TRADE POSITION   2023-07-27
20150.0 19400.0
condor         {'c1': {'strike': 20150.0, 'qty': -1000, 'cost': 113.4, 'final': 0.05, 'exp': datetime.date(2023, 8, 31)}, 'p1': {'strike': 19400.0, 'qty': -1000, 'cost': 108.85, 'final': 145.15, 'exp': datetime.date(2023, 8, 31)}}
premium        222250.0
hedge1       {'c1': {'strike': 20350.0, 'qty': 1000, 'cost': 4.3, 'final': 0.1, 'exp': datetime.date(2023, 8, 3)}, 'p1': {'strike': 19200.0, 'qty': 1000, 'cost': 8.3, 'final': 0.1, 'exp': datetime.date(2023, 8, 3)}}
hedge1_cost        12600.0
hedge1_pnl        -12400.0
premium        197250.0
hedge2       {'c1': {'strike': 20350.0, 'qty': 1000, 'cost': 0.1, 'final': 0.1, 'exp': datetime.date(2023, 8, 10)}, 'p1': {'strike': 19200.0, 'qty': 1000, '

In [74]:
df.get_shares_full()

2022-09-13 00:00:00+05:30    5565890048
2022-09-15 00:00:00+05:30    5566830080
2022-09-16 00:00:00+05:30    5566830080
2022-09-20 00:00:00+05:30    5569959936
2022-09-21 00:00:00+05:30    5569959936
                                ...    
2024-03-07 00:00:00+05:30    7596420096
2024-03-08 00:00:00+05:30    7596420096
2024-03-12 00:00:00+05:30    7693779968
2024-03-12 00:00:00+05:30    7596420096
2024-03-13 00:00:00+05:30    7596420096
Length: 150, dtype: int64